<a href="https://colab.research.google.com/github/Articbug/Telecom-CDR-Analytics-Platform/blob/main/Notebooks/2_CDR_Advanced_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
#   CDR ADVANCED SQL QUERIES
#   Wipro CDR Analytics Project
#   Author: Chandan Sahoo
# ============================================================

import subprocess, sys

packages = ['snowflake-connector-python', 'pandas']
for package in packages:
    subprocess.run(
        [sys.executable, '-m', 'pip', 'install', package, '--quiet', '--disable-pip-version-check'],
        capture_output=True
    )

print('All libraries installed successfully!')

import snowflake.connector
import pandas as pd

def get_connection():
    return snowflake.connector.connect(
        account  = 'bopsoxz-lr52214',
        user     = 'CHANDANSAHOO',
        password = 'Chandansahoosnowflake5',
        database = 'TELECOM_DWH',
        schema   = 'STAGING',
        warehouse= 'REPORTING_WH',
        role     = 'ACCOUNTADMIN'
    )

conn   = get_connection()
cursor = conn.cursor()
cursor.execute('SELECT CURRENT_USER(), CURRENT_ROLE(), CURRENT_DATABASE(), CURRENT_WAREHOUSE()')
row = cursor.fetchone()
print(f'Connected successfully!')
print(f'   User:      {row[0]}')
print(f'   Role:      {row[1]}')
print(f'   Database:  {row[2]}')
print(f'   Warehouse: {row[3]}')
conn.close()

All libraries installed successfully!
Connected successfully!
   User:      CHANDANSAHOO
   Role:      ACCOUNTADMIN
   Database:  TELECOM_DWH
   Warehouse: REPORTING_WH


In [ ]:
# ============================================================
#   CELL 2: ADVANCED SQL QUERIES
#   CTE, Window Functions, Fraud Detection, Network Analysis
# ============================================================
print('=' * 55)
print('   ADVANCED SQL ANALYTICS')
print('=' * 55)

conn   = get_connection()
cursor = conn.cursor()

# ── Query 1: Monthly Revenue Trend
print('\n1️⃣  Monthly Revenue Trend (CTE + Window Functions):')
cursor.execute('''
    WITH monthly_revenue AS (
        SELECT
            YEAR(CALL_START_TIME)           AS year_num,
            MONTH(CALL_START_TIME)          AS month_num,
            MONTHNAME(CALL_START_TIME)      AS month_name,
            COUNT(*)                        AS total_calls,
            ROUND(SUM(CHARGE_AMOUNT), 2)    AS total_revenue,
            COUNT(DISTINCT CALLING_NUMBER)  AS unique_customers
        FROM TELECOM_DWH.STAGING.STG_CDR
        GROUP BY YEAR(CALL_START_TIME), MONTH(CALL_START_TIME), MONTHNAME(CALL_START_TIME)
    ),
    revenue_with_growth AS (
        SELECT *,
            LAG(total_revenue) OVER (ORDER BY year_num, month_num) AS prev_revenue,
            ROUND((total_revenue - LAG(total_revenue) OVER (ORDER BY year_num, month_num))
                / NULLIF(LAG(total_revenue) OVER (ORDER BY year_num, month_num), 0) * 100, 2) AS mom_growth_pct,
            ROUND(total_revenue / SUM(total_revenue) OVER () * 100, 2) AS revenue_share_pct
        FROM monthly_revenue
    )
    SELECT month_name, total_calls, unique_customers,
           total_revenue, prev_revenue, mom_growth_pct, revenue_share_pct
    FROM revenue_with_growth
    ORDER BY year_num, month_num
''')
rows  = cursor.fetchall()
cols  = [d[0] for d in cursor.description]
df_q1 = pd.DataFrame(rows, columns=cols)
print(df_q1.to_string(index=False))

# ── Query 2: Customer Ranking
print('\n2️⃣  Top 10 Customers by Revenue (RANK + NTILE):')
cursor.execute('''
    WITH customer_revenue AS (
        SELECT CALLING_NUMBER, COUNT(*) AS total_calls,
               ROUND(SUM(CHARGE_AMOUNT), 2) AS total_revenue,
               COUNT(DISTINCT CALL_TYPE) AS services_used
        FROM TELECOM_DWH.STAGING.STG_CDR
        GROUP BY CALLING_NUMBER
    )
    SELECT CALLING_NUMBER, total_calls, total_revenue, services_used,
           RANK() OVER (ORDER BY total_revenue DESC) AS revenue_rank,
           CASE NTILE(4) OVER (ORDER BY total_revenue DESC)
               WHEN 1 THEN 'PLATINUM' WHEN 2 THEN 'GOLD'
               WHEN 3 THEN 'SILVER'   ELSE 'BRONZE' END AS tier
    FROM customer_revenue
    ORDER BY revenue_rank LIMIT 10
''')
rows  = cursor.fetchall()
cols  = [d[0] for d in cursor.description]
df_q2 = pd.DataFrame(rows, columns=cols)
print(df_q2.to_string(index=False))

# ── Query 3: Fraud Detection
print('\n3️⃣  Fraud Detection (Rolling 7-Day Average):')
cursor.execute('''
    WITH daily_stats AS (
        SELECT DATE(CALL_START_TIME) AS call_date,
               COUNT(*) AS daily_calls,
               SUM(CASE WHEN IS_FRAUD = TRUE THEN 1 ELSE 0 END) AS fraud_calls,
               ROUND(SUM(CHARGE_AMOUNT), 2) AS daily_revenue
        FROM TELECOM_DWH.STAGING.STG_CDR
        GROUP BY DATE(CALL_START_TIME)
    )
    SELECT call_date, daily_calls, fraud_calls, daily_revenue,
           ROUND(AVG(daily_calls) OVER (
               ORDER BY call_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW), 1) AS rolling_7day_calls,
           ROUND(AVG(fraud_calls) OVER (
               ORDER BY call_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW), 2) AS rolling_7day_fraud
    FROM daily_stats
    ORDER BY call_date LIMIT 15
''')
rows  = cursor.fetchall()
cols  = [d[0] for d in cursor.description]
df_q3 = pd.DataFrame(rows, columns=cols)
print(df_q3.to_string(index=False))

# ── Query 4: Network Performance
print('\n4️⃣  Network Performance Analysis:')
cursor.execute('''
    SELECT NETWORK_TYPE, CALL_TYPE, COUNT(*) AS total_calls,
           ROUND(AVG(DURATION_SECONDS), 1) AS avg_duration,
           ROUND(SUM(CHARGE_AMOUNT), 2) AS total_revenue,
           ROUND(SUM(CASE WHEN TERMINATION_CD = 'DROPPED' THEN 1 ELSE 0 END)
                 * 100.0 / COUNT(*), 2) AS drop_rate_pct,
           RANK() OVER (PARTITION BY CALL_TYPE ORDER BY
               SUM(CASE WHEN TERMINATION_CD = 'DROPPED' THEN 1 ELSE 0 END)
               * 100.0 / COUNT(*) ASC) AS reliability_rank
    FROM TELECOM_DWH.STAGING.STG_CDR
    GROUP BY NETWORK_TYPE, CALL_TYPE
    ORDER BY CALL_TYPE, reliability_rank
''')
rows  = cursor.fetchall()
cols  = [d[0] for d in cursor.description]
df_q4 = pd.DataFrame(rows, columns=cols)
print(df_q4.to_string(index=False))

# ── Query 5: Cell Tower Performance
print('\n5️⃣  Cell Tower Performance:')
cursor.execute('''
    SELECT CELL_ID, COUNT(*) AS total_calls,
           ROUND(AVG(DURATION_SECONDS), 1) AS avg_duration,
           ROUND(SUM(CHARGE_AMOUNT), 2) AS total_revenue,
           SUM(CASE WHEN TERMINATION_CD = 'DROPPED' THEN 1 ELSE 0 END) AS dropped_calls,
           ROUND(SUM(CASE WHEN TERMINATION_CD = 'DROPPED' THEN 1 ELSE 0 END)
                 * 100.0 / COUNT(*), 2) AS drop_rate_pct,
           SUM(CASE WHEN IS_FRAUD = TRUE THEN 1 ELSE 0 END) AS fraud_calls,
           RANK() OVER (ORDER BY SUM(CHARGE_AMOUNT) DESC) AS revenue_rank
    FROM TELECOM_DWH.STAGING.STG_CDR
    GROUP BY CELL_ID
    ORDER BY revenue_rank
''')
rows  = cursor.fetchall()
cols  = [d[0] for d in cursor.description]
df_q5 = pd.DataFrame(rows, columns=cols)
print(df_q5.to_string(index=False))

cursor.close()
conn.close()
print('\n✅ All Advanced SQL Queries Complete!')

   ADVANCED SQL ANALYTICS

1️⃣  Monthly Revenue Trend (CTE + Window Functions):
MONTH_NAME  TOTAL_CALLS  UNIQUE_CUSTOMERS TOTAL_REVENUE PREV_REVENUE MOM_GROWTH_PCT REVENUE_SHARE_PCT
       Jan         4265               499       5052.67         None           None              8.51
       Feb         4064               500       4817.58      5052.67          -4.65              8.11
       Mar         4212               499       5001.30      4817.58           3.81              8.42
       Apr         4042               500       4736.15      5001.30          -5.30              7.97
       May         4260               500       5094.61      4736.15           7.57              8.58
       Jun         4115               500       4892.69      5094.61          -3.96              8.24
       Jul         4189               500       4979.05      4892.69           1.77              8.38
       Aug         4128               500       5094.25      4979.05           2.31              8.58
  